# Whisper Prompts Testing

### Initializing

In [1]:
import os
import re
import glob

In [2]:
import whisper
import pandas as pd

/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [3]:
model = whisper.load_model('large-v2')

### Reading WAV and TXT files

In [40]:
# WAV files
first_file = 0
num_files = 5

last_file = first_file+num_files

wav_atco2_asr = glob.glob('./WhisperModel/ATCO2-ASR/DATA/*.wav')
wav_atco2_asr.sort()
wav_atco2_asr = wav_atco2_asr[first_file:last_file]

wav_atcosim   = glob.glob('./WhisperModel/ATCOSIM/WAVdata/*/*/*.wav')
wav_atcosim.sort()
wav_atcosim = wav_atcosim[first_file:last_file]

wav_zcu       = glob.glob('./WhisperModel/ZCU_CZ_ATC/*.wav')
wav_zcu.sort()
wav_zcu = wav_zcu[first_file:last_file]

# Transcripts
trs_atco2_asr = glob.glob('./WhisperModel/ATCO2-ASR/DATA/*.txt')
trs_atco2_asr.sort()
trs_atco2_asr = trs_atco2_asr[first_file:last_file]

trs_atcosim   = glob.glob('./WhisperModel/ATCOSIM/TXTdata/*/*/*.filtered.txt')
trs_atcosim.sort()
trs_atcosim = trs_atcosim[first_file:last_file]

trs_zcu       = glob.glob('./WhisperModel/ZCU_CZ_ATC/*.txt')
trs_zcu.sort()
trs_zcu = trs_zcu[first_file:last_file]

# Print out to make sure the audio and text files match
print(wav_atco2_asr)
print(trs_atco2_asr)

print(wav_atcosim)
print(trs_atcosim)

print(wav_zcu)
print(trs_zcu)

['./WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_091112.wav', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_120512.wav', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_121325.wav', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_130407.wav', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_140929.wav']
['./WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_091112.txt', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_120512.txt', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_121325.txt', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_130407.txt', './WhisperModel/ATCO2-ASR/DATA/LKPR_RUZYNE_Radar_120_520MHz_20201025_140929.txt']
['./WhisperModel/ATCOSIM/WAVdata/gf1/gf1_01/gf1_01_001.wav', './WhisperModel/ATCOSIM/WAVdata/gf1/gf1_01/gf1_01_002.wav', './WhisperModel/ATCOSIM/WAVdata/gf1/gf1_01/gf1_01_003.wav

### Setting up the prompts

In [41]:
prompt_general = "Air Traffic Control communications"

airlines_atco2 = "Ryanair, Eurowings"
airlines_atcosim = "Swissair, Cross Air, Luxair, Lufthansa"
airlines_zcu = "Lufthansa, Wizzair, Niki, Condor, Aeroflot, Malaysian, Scandinavian"
airlines = airlines_atco2 + ", " + airlines_atcosim + ", " + airlines_zcu

waypoints_atco2 = "Baltu"
waypoints_atcosim = ""
waypoints_zcu = "Lanux, Odnem, Rapet, Pemur, Lagar"
waypoints = waypoints_atco2 + ", " + waypoints_atcosim + ", " + waypoints_zcu

entities_atco2 = "Praha radar, Prague, Praha"
entities_atcosim = "Geneva, Marseille, Kines Saint Prex, Willisau, Torino, Passeiry"
entities_zcu = "Munich, Rhein, Bratislava, Prague, Praha"
entities = entities_atco2 + ", " + entities_atcosim + ", " + entities_zcu

nato = "alpha,bravo,charlie,delta,echo,foxtrot,golf,hotel,india,juliett,kilo,lima,mike,november,oscar,papa,quebec,romeo,sierra,tango,uniform,victor,whiskey,xray,yankee,zulu"
terminology = "climb, climbing, descend, descending, passing, feet, knots, degrees, direct, maintain, identified, ILS, VFR, IFR, contact, frequency, turn, right, left, heading, altitude, flight, level, cleared, squawk, approach, runway, established, report, affirm, negative, wilco, roger, radio, radar"

prompt_atco2_asr = prompt_general + " " + airlines_atco2 + " " + waypoints_atco2 + " " + entities_atco2 + " " + nato + " " + terminology
prompt_atcosim = prompt_general + " " + airlines_atcosim + " " + waypoints_atcosim + " " + entities_atcosim + " " + nato + " " + terminology
prompt_zcu = prompt_general + " " + airlines_zcu + " " + waypoints_zcu + " " + entities_zcu + " " + nato + " " + terminology

# prompt = prompt_general + " " + airlines + " " + waypoints + " " + entities + " " + nato + " " + terminology
# prompt_atco2_asr = prompt
# prompt_atcosim = prompt
# prompt_zcu = prompt

print(prompt_atco2_asr)
print(prompt_atcosim)
print(prompt_zcu)

Air Traffic Control communications Ryanair, Eurowings Baltu Praha radar, Prague, Praha alpha,bravo,charlie,delta,echo,foxtrot,golf,hotel,india,juliett,kilo,lima,mike,november,oscar,papa,quebec,romeo,sierra,tango,uniform,victor,whiskey,xray,yankee,zulu climb, climbing, descend, descending, passing, feet, knots, degrees, direct, maintain, identified, ILS, VFR, IFR, contact, frequency, turn, right, left, heading, altitude, flight, level, cleared, squawk, approach, runway, established, report, affirm, negative, wilco, roger, radio, radar
Air Traffic Control communications Swissair, Cross Air, Luxair, Lufthansa  Geneva, Marseille, Kines Saint Prex, Willisau, Torino, Passeiry alpha,bravo,charlie,delta,echo,foxtrot,golf,hotel,india,juliett,kilo,lima,mike,november,oscar,papa,quebec,romeo,sierra,tango,uniform,victor,whiskey,xray,yankee,zulu climb, climbing, descend, descending, passing, feet, knots, degrees, direct, maintain, identified, ILS, VFR, IFR, contact, frequency, turn, right, left, hea

### Running Whisper on the files, W/ and W/O prompt

#### ATCO2 - ASR

In [ ]:
result_prompt_atco2_asr = []
result_clean_atco2_asr = []
correct_atco2_asr = []

print('ATCO2 - ASR')
for i in range(len(wav_atco2_asr)):
    result = model.transcribe(wav_atco2_asr[i], initial_prompt=prompt_atco2_asr, language='en')
    result_prompt_atco2_asr.append(result['text'])

    result = model.transcribe(wav_atco2_asr[i], language='en')
    result_clean_atco2_asr.append(result['text'])
    
    with open(trs_atco2_asr[i], 'r') as f:
        correct = f.read()
        correct_atco2_asr.append(correct)
    print('Whisper:'+result['text'])
    print('Correct: '+correct)

ATCO2 - ASR


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: Oscar Kilo Papa Mike Bravo, descend FL100 Descend FL100, Oscar Kilo Papa Mike Bravo
Correct: Oscar Kilo Papa Mike Bravo descend flight level one hundred level one hundred Oscar Kilo Papa Mike Bravo 


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: OSCQA, QNH 100, IFR flight starts now, time 05, cleared to destination via FL10R
Correct: Oscar Kilo Kilo Echo Alfa Praha Radar identified climb flight level one hundred IFR flight starts [hes] now time zero five cleared to destination via flight plan route 


...

Whisper: Ryanair 730H turn left heading 360 Go Ryanair 730H
Correct: Ryanair Seven Three Alpha Hotel turn left heading three six zero Ryanair Seven Three Alpha Hotel 


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: Oskar, Kilo Kilo Info, November, proceed direct BALTU. Proceed direct BALTU, Oskar, Kilo Kilo Info, November.
Correct: Oscar Kilo Kilo Uniform November proceed direct Baltu proceed direct Baltu Oscar Kilo Kilo Uniform November 


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: EW7AB, turn right heading 210, cleared ILS approach runway 24, report established. James 210, cleared for the ILS 24, report established, EW7AB.
Correct: Eurowings Seven Alfa Bravo turn right heading two one zero cleared ILS approach runway two four report established heading(-ding) [hes] two one zero cleared for the ILS two four report established Eurowings Seven Alpha Bravo 


#### ATCOSIM

In [42]:
result_prompt_atcosim = []
result_clean_atcosim = []
correct_atcosim = []

print('ATCOSIM')
for i in range(len(wav_atcosim)):
    result = model.transcribe(wav_atcosim[i], initial_prompt=prompt_atcosim, language='en')
    result_prompt_atcosim.append(result['text'])

    result = model.transcribe(wav_atcosim[i], language='en')
    result_clean_atcosim.append(result['text'])
    
    with open(trs_atcosim[i], 'r') as f:
        correct = f.read()
        correct_atcosim.append(correct)
    print('Whisper:'+result['text'])
    print('Correct: '+correct)


ATCOSIM
Whisper: G-C Geneva 128.15, goodbye.
Correct:  contact geneva one two eight decimal one five good bye 
Whisper: Sucerne 660R contact Marseille 125.85, goodbye.
Correct: swissair six six zero romeo contact marseille one two five decimal eight five good bye 
Whisper: Crossfire 512, bonjour, Identified, cleared QNH 100, privileged off, flight level 300.
Correct: cross air five one two bonjour identified cleared kines st prex willisau flight level three zero zero 
Whisper: Bonjour, Luxair 551 identified, cleared to pass through Torino, FL330.
Correct: bonjour luxair five five one identified cleared passeiry torino flight level three three zero 
Whisper: Lufthansa 4723 resume on navigation to Vélizard
Correct: lufthansa four seven two three resume own navigation to willisau 


#### ZCU

In [ ]:
result_prompt_zcu = []
result_clean_zcu = []
correct_zcu = []

print('ZCU')
for i in range(len(wav_zcu)):
    result = model.transcribe(wav_zcu[i], initial_prompt=prompt_zcu, language='en')
    result_prompt_zcu.append(result['text'])

    result = model.transcribe(wav_zcu[i], language='en')
    result_clean_zcu.append(result['text'])
    
    with open(trs_zcu[i], 'r') as f:
        correct = f.read()
        correct_zcu.append(correct)
    print('Whisper:'+result['text'])
    print('Correct: '+correct)

ZCU


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: Lufthansa 4JAL, contact Munich 132.550, good bye. Lufthansa 4JAL, 132.550, see you later, thank you and good bye. Praha, QNH 827, are you there? Go. Yeah, just confirm the frequency please. 127.425 Okay, we try again on 127.425 but we get no reply for the moment, but we try again, thank you. Praha radar, WZPR, good morning, climbing FL240, 223. WZPR, Praha radar, contact squawk 1471, climb FL340. Squawk 1471, climb FL340, WZPR. Praha radar, good morning, contact 8TW, FL230, direct OSCE, KDAG. Contact 8TW, Praha radar, contact direct LK. Direct LK, contact 8TW. QNH 79, WZPR, 134.475. Praha radar, good morning, N8016, passing FL170, climb FL240, inbound to Lanox. N8016, Praha radar, contact. QNH 79, Praha. Yeah, QNH 79, we get with you no reply on 127.425. Yeah, right, frequency is 134.475, bye bye. 134.475, bye bye, QNH 79. Condor 042, direct to Odenham. Condor 042, present position direct to Odenham. Odenham. N8016, climb FL340. Climb FL340, N8016.
Correct: Lufthansa 4 J A con

/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: Lufthansa 739 Praha? Lufthansa 739 just with you, descending down level 350. Good afternoon Lufthansa 739 Praha, radar contact, descend FL300, level by rapid. Lufthansa 739 descending FL300, level, laser to carpet.
Correct: Lufthansa 7 3 9 Praha Lufthansa 7 3 9  radio descending down level 3 5 0 good afternoon Lufthansa 7 3 9 Praha radar contact descend FL 3 0 0 level by RAPET Lufthansa 7 3 9 descending FL 3 0 0 level  


...

Whisper: Radar, Aerofort 122, good afternoon, passing FL268 for FL280. Aerofort 122, hello, radar contact proceed direct TEMUR, climb to FL310. Climb to FL310, clear to direct TEMUR, Aerofort 122. Aerofort 122, hello, radar contact, maintain FL330. Climb to FL330, Malaysian 21.
Correct: Radar Aeroflot 1 2 2 good afternoon passing FL 2 8 climb FL 2 8 0 Aeroflot 1 2 2 hello radar contact proceed to PEMUR climb to FL 3 1 0 climbing level 3 1 0 cleared to direct PEMUR Aeroflot 1 2 2 Praha good afternoon Malaysian 2 1 FL 3 3 0 squawking 0 6 4 7 Malaysian 2 1 hello radar contact maintain FL 3 3 0 level 3 3 0 Malaysian 2 1 


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: Scandinavian 1665 contact Praha 135.135 goodbye. 35.135 Scandinavian 1665 bye.
Correct: Scandinavian 1 6 6 5 contact Praha 1 3 5 . 1 3 5 good bye  3 5 1 3 5 Scandinavian 1 6 6  


/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper: PAPA Radar, Houston 2550, passing level 233, climbing level 280. Houston 2550, hello radar, contact climb to flight level 330. Houston 2550, climb level 330. GULFIN 789, contact Praha 135, 135, TVR. Over to Praha 135, 135, GULFIN 789, next runway. Lufthansa 77P, contact Praha 136, 405, bye bye. Lufthansa 77P, 136, 405, thank you, bye bye.
Correct: Praha Radar Lufthansa 2 5 5 0 passing level 2 3 3 climbing level 2 8 0 Lufthansa 2 5 5 0 hello radar contact climb to FL 3 3 0 Lufthansa 2 5 5 0 climb level 3 3 0 7 8 9 contact Praha 1 3 5 1 3 5  (Praha(Prague)) 1 3 5 1 3 5  7 9  Lufthansa 7 7 P contact (Rhein(rajn)) 1 3 6 4 0 5 bye bye Lufthansa 7 7 P 1 3 64 0 5 thank you bye bye 


### Framing the data

In [43]:
data = pd.DataFrame(columns=['ATCO2-ASR-whisper-clean', 'ATCO2-ASR-whisper-prompt','ATCO2-ASR-correct', 'ATCOSIM-whisper-clean', 'ATCOSIM-whisper-prompt', 'ATCOSIM-correct', 'ZCU-whisper-clean', 'ZCU-whisper-prompt', 'ZCU-correct'])

In [45]:
# data['ATCO2-ASR-whisper-clean'] = result_clean_atco2_asr
# data['ATCO2-ASR-whisper-prompt'] = result_prompt_atco2_asr
# data['ATCO2-ASR-correct'] = correct_atco2_asr

data['ATCOSIM-whisper-clean'] = result_clean_atcosim
data['ATCOSIM-whisper-prompt'] = result_prompt_atcosim
data['ATCOSIM-correct'] = correct_atcosim

# data['ZCU-whisper-clean'] = result_clean_zcu
# data['ZCU-whisper-prompt'] = result_prompt_zcu
# data['ZCU-correct'] = correct_zcu

# data.T.to_excel('Whisper-Prompt-test.xlsx') # data is saved with file indexes as columns and data labels as row indexes for easier reading

In [46]:
try:
    data
except:
    import pandas as pd
    import numpy as np
    data = pd.read_excel('Whisper-Prompt-test.xlsx', header=None, index_col=0)
    data = data.T.drop(columns=np.nan)
    data.columns.name = None
    data = data.reset_index().drop(columns='index')

# data_atco2_asr = data[['ATCO2-ASR-whisper-clean', 'ATCO2-ASR-whisper-prompt', 'ATCO2-ASR-correct']]
# data_atco2_asr = data_atco2_asr.rename(columns={'ATCO2-ASR-whisper-clean': 'hyp-clean', 'ATCO2-ASR-whisper-prompt': 'hyp-prmpt', 'ATCO2-ASR-correct': 'ref'})

data_atcosim   = data[['ATCOSIM-whisper-clean', 'ATCOSIM-whisper-prompt', 'ATCOSIM-correct']]
data_atcosim   = data_atcosim.rename(columns={'ATCOSIM-whisper-clean': 'hyp-clean', 'ATCOSIM-whisper-prompt': 'hyp-prmpt', 'ATCOSIM-correct': 'ref'})

# data_zcu       = data[['ZCU-whisper-clean', 'ZCU-whisper-prompt', 'ZCU-correct']]
# data_zcu       = data_zcu.rename(columns={'ZCU-whisper-clean': 'hyp-clean', 'ZCU-whisper-prompt': 'hyp-prmpt', 'ZCU-correct': 'ref'})

### Filtering and Normalizing

In [ ]:
import sys
import os
current = os.path.dirname(os.path.realpath(__file__))
parent = os.path.dirname(current)
sys.path.append(parent+'/Evaluate')
from Normalizer import filterAndNormalize, normalizeOnly

##### Transform

In [49]:
# data_atco2_asr.loc[:, 'hyp-clean-norm'] = data_atco2_asr.apply(lambda x: filterAndNormalize(x['hyp-clean']), axis=1)
# data_atco2_asr.loc[:, 'hyp-prmpt-norm'] = data_atco2_asr.apply(lambda x: filterAndNormalize(x['hyp-prmpt']), axis=1)
# data_atco2_asr.loc[:, 'ref-norm']       = data_atco2_asr.apply(lambda x: filterAndNormalize(x['ref']), axis=1)

data_atcosim.loc[:, 'hyp-clean-norm']   = data_atcosim.apply(lambda x: normalizeOnly(x['hyp-clean']), axis=1)
data_atcosim.loc[:, 'hyp-prmpt-norm']   = data_atcosim.apply(lambda x: normalizeOnly(x['hyp-prmpt']), axis=1)
data_atcosim.loc[:, 'ref-norm']         = data_atcosim.apply(lambda x: normalizeOnly(x['ref']), axis=1)

# data_zcu.loc[:, 'hyp-clean-norm']       = data_zcu.apply(lambda x: filterAndNormalize(x['hyp-clean']), axis=1)
# data_zcu.loc[:, 'hyp-prmpt-norm']       = data_zcu.apply(lambda x: filterAndNormalize(x['hyp-prmpt']), axis=1)
# data_zcu.loc[:, 'ref-norm']             = data_zcu.apply(lambda x: filterAndNormalize(x['ref']), axis=1)


# # Creating Excel Writer Object from Pandas  
# writer = pd.ExcelWriter('Transcripts.xlsx', engine='xlsxwriter')   
# data_atco2_asr.T.to_excel(excel_writer=writer, sheet_name='ATCO2-ASR')
# data_atcosim.T.to_excel(writer, sheet_name='ATCOSIM')
# data_zcu.T.to_excel(writer, sheet_name='ZCU')
# writer.save()

### Calculate WER

In [50]:
import jiwer

In [53]:
# WER ( reference (correct) , hypothesis (whisper) )
# wer_atco2_asr_clean  = jiwer.wer(list(data_atco2_asr['ref-norm']), list(data_atco2_asr['hyp-clean-norm']))
# wer_atco2_asr_prompt = jiwer.wer(list(data_atco2_asr['ref-norm']), list(data_atco2_asr['hyp-prmpt-norm']))

# data_atcosim.loc[0, 'ref-norm'] = 'empty'
# data_atcosim.loc[3, 'ref-norm'] = 'emtpy'
# data_atcosim.loc[6, 'ref-norm'] = 'empty'
wer_atcosim_clean  = jiwer.wer(list(data_atcosim['ref-norm']), list(data_atcosim['hyp-clean-norm']))
wer_atcosim_prompt = jiwer.wer(list(data_atcosim['ref-norm']), list(data_atcosim['hyp-prmpt-norm']))

# wer_zcu_clean  = jiwer.wer(list(data_zcu['ref-norm']), list(data_zcu['hyp-clean-norm']))
# wer_zcu_prompt = jiwer.wer(list(data_zcu['ref-norm']), list(data_zcu['hyp-prmpt-norm']))

In [54]:
import numpy as np
# print('ATCO2 ASR -- no prompt: {} %'.format(round(wer_atco2_asr_clean*100,2)))
# print('ATCO2 ASR --    prompt: {} %'.format(round(wer_atco2_asr_prompt*100,2)))
print()
print('ATCOSIM   -- no prompt: {} %'.format(round(wer_atcosim_clean*100,2)))
print('ATCOSIM   --    prompt: {} %'.format(round(wer_atcosim_prompt*100,2)))
print()
# print('ZCU       -- no prompt: {} %'.format(round(wer_zcu_clean*100,2)))
# print('ZCU       --    prompt: {} %'.format(round(wer_zcu_prompt*100,2)))
# print()

# wer_avg_clean = np.average([wer_atco2_asr_clean, wer_atcosim_clean, wer_zcu_clean])
# wer_avg_prompt = np.average([wer_atco2_asr_prompt, wer_atcosim_prompt, wer_zcu_prompt])
# print('Average   -- no prompt: {} %'.format(round(wer_avg_clean*100,2)))
# print('Average   --    prompt: {} %'.format(round(wer_avg_prompt*100,2)))


ATCOSIM   -- no prompt: 57.45 %
ATCOSIM   --    prompt: 27.66 %

